In [1]:
import pandas as pd
import numpy as np
import os
import re
from pykospacing import Spacing
from konlpy.tag import Hannanum,Kkma,Komoran

In [3]:
path = './data/공시_다트크롤링/'
file_list = os.listdir(path)
file_list

['AJ네트웍스.csv',
 'AK홀딩스.csv',
 'BGF.csv',
 'BGF리테일.csv',
 'BNK금융지주.csv',
 'BYC.csv',
 'CJ CGV.csv',
 'CJ.csv',
 'CJ대한통운.csv',
 'CJ씨푸드.csv',
 'CJ제일제당.csv',
 'CS홀딩스.csv',
 'DB.csv',
 'DB금융투자.csv',
 'DB손해보험.csv',
 'DB하이텍.csv',
 'DGB금융지주.csv',
 'DL.csv',
 'DL건설.csv',
 'DL이앤씨.csv',
 'DRB동일.csv',
 'DSR.csv',
 'DSR제강.csv',
 'E1.csv',
 'ESR켄달스퀘어리츠.csv',
 'F&F 홀딩스.csv',
 'F&F.csv',
 'GKL.csv',
 'GS.csv',
 'GS건설.csv',
 'GS글로벌.csv',
 'GS리테일.csv',
 'HDC.csv',
 'HDC랩스.csv',
 'HDC현대EP.csv',
 'HDC현대산업개발.csv',
 'HD현대.csv',
 'HJ중공업.csv',
 'HLB글로벌.csv',
 'HMM.csv',
 'HSD엔진.csv',
 'JB금융지주.csv',
 'JW생명과학.csv',
 'JW중외제약.csv',
 'JW홀딩스.csv',
 'KB금융.csv',
 'KC그린홀딩스.csv',
 'KC코트렐.csv',
 'KEC.csv',
 'KG스틸.csv',
 'KG케미칼.csv',
 'KH 필룩스.csv',
 'KISCO홀딩스.csv',
 'KPX케미칼.csv',
 'KPX홀딩스.csv',
 'KR모터스.csv',
 'KSS해운.csv',
 'KTcs.csv',
 'KTis.csv',
 'LF.csv',
 'LG.csv',
 'LG디스플레이.csv',
 'LG생활건강.csv',
 'LG에너지솔루션.csv',
 'LG유플러스.csv',
 'LG이노텍.csv',
 'LG전자.csv',
 'LG헬로비전.csv',
 'LG화학.csv',
 'LIG넥스원.csv',
 'LS.csv',
 'LS네트웍스.c

In [48]:
file_path = os.path.join(path,file_list[94])
df = pd.read_csv(file_path,index_col=0)

In [49]:
df = df[['접수일자','공시대상회사','보고서명']]
df.columns = ['날짜','공시회사','공시보고서명']
df['날짜'] = pd.to_datetime(df['날짜'])
df['날짜'] = df['날짜'].apply(lambda x : x.to_period(freq='D'))

In [50]:
df.head()

,날짜,공시회사,공시보고서명
0,2022-05-26,유 SK,증권신고서(채무증권)
1,2022-05-24,유 SK,타법인주식및출자증권취득결정 (자회사의 주요경영사항)
2,2022-05-16,유 SK,분기보고서 (2022.03)
3,2022-04-29,유 SK,해외증권거래소등에신고한사업보고서등의국내신고 (자회사의 주요경영사항)
4,2022-04-28,유 SK,현금ㆍ현물배당결정 (자회사의 주요경영사항)


In [51]:
df.drop_duplicates(subset=['공시보고서명'],inplace=True)
df = df.sort_values(by='날짜')
df

,날짜,공시회사,공시보고서명
1,2016-01-21,유 SK,주요사항보고서(회사합병결정) (자회사의 주요경영사항)
6,2016-02-12,유 SK,기타안내사항(안내공시)
0,2016-03-10,유 SK,[기재정정]매출액또는손익구조30%(대규모법인은15%)이상변경
12,2016-03-14,유 SK,[기재정정]매출액또는손익구조30%(대규모법인은15%)이상변경 (자회사의 주요경영사항)
4,2016-05-10,유 SK,[기재정정]해외증권거래소등에신고한사업보고서등의국내신고 (자회사의 주요경영사항)
...,...,...,...
4,2022-04-28,유 SK,현금ㆍ현물배당결정 (자회사의 주요경영사항)
3,2022-04-29,유 SK,해외증권거래소등에신고한사업보고서등의국내신고 (자회사의 주요경영사항)
2,2022-05-16,유 SK,분기보고서 (2022.03)
1,2022-05-24,유 SK,타법인주식및출자증권취득결정 (자회사의 주요경영사항)


In [53]:
df['공시보고서명'][:50]

1                         주요사항보고서(회사합병결정) (자회사의 주요경영사항)
6                                          기타안내사항(안내공시)
0                     [기재정정]매출액또는손익구조30%(대규모법인은15%)이상변경
12      [기재정정]매출액또는손익구조30%(대규모법인은15%)이상변경 (자회사의 주요경영사항)
4           [기재정정]해외증권거래소등에신고한사업보고서등의국내신고 (자회사의 주요경영사항)
3                                 [기재정정]사업보고서 (2015.12)
2                                 [기재정정]분기보고서 (2016.03)
3                                       반기보고서 (2016.06)
3                                       분기보고서 (2016.09)
0     [기재정정]타법인주식및출자증권취득결정 (자회사의 주요경영사항)  (자회사인 SK종합...
13                             조회공시요구(풍문또는보도)에대한답변(미확정)
13                                       [기재정정]주주총회소집결의
11                                      사업보고서 (2016.12)
4                                       분기보고서 (2017.03)
7                                         기타시장안내 (업종변경)
4                             단일판매ㆍ공급계약체결 (자회사의 주요경영사항)
2                                      풍문또는보도에대한해명(미확정)
6                                  영업양수결정 (자회사의 

In [55]:
df['공시보고서명'].value_counts().nunique()

1

In [62]:
df['공시보고서명'] = df['공시보고서명'].apply(spacing)

In [63]:
df['공시보고서명'][:30]

1                    주요 사항 보고서(회사 합병 결정) (자회사의 주요 경영사항)
6                                         기타 안내사항(안내공시)
0             [기재정정] 매출액 또는 손익구조 30%(대규모 법인은 15%) 이상 변경
12    [기재정정] 매출액 또는 손익구조 30%(대규모 법인은 15%) 이상 변경 (자회사...
4     [기재정정] 해외 증권거래소 등에 신고한 사업보고서 등의 국내 신고 (자회사의 주요...
3                                [기재정정] 사업보고서 (2015.12)
2                                 [기재정정]분기보고서 (2016.03)
3                                       반기보고서 (2016.06)
3                                       분기보고서 (2016.09)
0     [기재정정] 타법인 주식 및 출자증권 취득 결정 (자회사의 주요 경영사항) (자회사...
13                        조회공시 요구(풍문 또는 보도)에 대한 답변(미확정)
13                                    [기재정정] 주주총회 소집 결의
11                                      사업보고서 (2016.12)
4                                       분기보고서 (2017.03)
7                                       기타 시장 안내 (업종변경)
4                       단일 판매 ㆍ 공급 계약 체결 (자회사의 주요 경영사항)
2                                  풍문 또는 보도에 대한 해명(미확정)
6                                영업양수 결정 (자회사의 주

In [58]:
han = Hannanum()
kom = Komoran()
kkm = Kkma()

In [64]:
df['공시보고서명'].apply(han.nouns)[:30]

1               [주요, 사항, 보고서(회사, 합병, 결정, 자회사, 주요, 경영사항]
6                                      [기타, 안내사항, 안내공시]
0            [기재정정, 매출액, 손익구조, 30, 대규모, 법인, 15, 이상, 변경]
12    [기재정정, 매출액, 손익구조, 30, 대규모, 법인, 15, 이상, 변경, 자회사...
4     [기재정정, 해외, 증권거래소, 등, 사업보고서, 등, 국내, 자회사, 주요, 경영사항]
3                               [기재정정, 사업보고, 2015., 12]
2                               [기재정정, 분기보고, 2016., 03]
3                                     [반기보고, 2016., 06]
3                                     [분기보고, 2016., 09]
0     [기재정정, 법, 주식, 출자증권, 취득, 결정, 자회사, 주요, 경영사항, 자회사...
13                            [조회공시, 요구(풍문, 보도, 답변(미확정]
13                                 [기재정정, 주주총회, 소집, 결의]
11                                   [사업보고서, 2016., 12]
4                                     [분기보고, 2017., 03]
7                                     [기타, 시장, 내, 업종변경]
4                 [단, 판매, ㆍ, 공급, 계약, 체결, 자회사, 주요, 경영사항]
2                                      [풍문, 보도, 해명(미확정]
6                             [영업양수, 결정, 자회사, 주요

In [65]:
df['공시보고서명'].apply(kom.nouns)[:30]

1                    [사항, 보고서, 회사, 합병, 결정, 자회사, 경영, 사항]
6                                  [기타, 안내, 사항, 안내, 공시]
0                 [기재, 정정, 매출액, 손익, 구조, 규모, 법인, 이상, 변경]
12    [기재, 정정, 매출액, 손익, 구조, 규모, 법인, 이상, 변경, 자회사, 경영,...
4     [기재, 정정, 해외, 증권거래소, 등, 신고, 사업, 보고서, 등, 국내, 신고,...
3                                     [기재, 정정, 사업, 보고서]
2                                     [기재, 정정, 분기, 보고서]
3                                             [반기, 보고서]
3                                             [분기, 보고서]
0     [기재, 정정, 법인, 주식, 출자, 증권, 취득, 결정, 자회사, 경영, 사항, ...
13                      [조회, 공시, 요구, 풍문, 보도, 답변, 미, 확정]
13                               [기재, 정정, 주주총회, 소집, 결의]
11                                            [사업, 보고서]
4                                             [분기, 보고서]
7                                  [기타, 시장, 안내, 업종, 변경]
4                     [단일, 판매, 공급, 계약, 체결, 자회사, 경영, 사항]
2                                   [풍문, 보도, 해명, 미, 확정]
6                             [영업, 양수, 결정, 자회사, 

In [66]:
df['공시보고서명'].apply(kkm.nouns)[:30]

1              [주요, 사항, 보고서, 회사, 합병, 결정, 자회사, 경영, 경영사항]
6                          [기타, 안내, 안내사항, 사항, 안내공시, 공시]
0     [기, 기재정정, 재정, 정, 매출액, 손익, 손익구조, 구조, 30, 대규모, 법...
12    [기, 기재정정, 재정, 정, 매출액, 손익, 손익구조, 구조, 30, 대규모, 법...
4     [기, 기재정정, 재정, 정, 해외, 증권, 증권거래소, 거래소, 등에, 신고, 사...
3             [기, 기재정정, 재정, 정, 사업, 사업보고서, 보고서, 2015.12]
2             [기, 기재정정, 재정, 정, 분기, 분기보고서, 보고서, 2016.03]
3                             [반기, 반기보고서, 보고서, 2016.06]
3                             [분기, 분기보고서, 보고서, 2016.09]
0     [기, 기재정정, 재정, 정, 법인, 주식, 출자, 출자증권, 증권, 취득, 결정,...
13     [조회, 조회공시, 공시, 요구, 풍, 풍문, 문, 보도, 답변, 미, 미확정, 확정]
13               [기, 기재정정, 재정, 정, 주주, 주주총회, 총회, 소집, 결의]
11                            [사업, 사업보고서, 보고서, 2016.12]
4                             [분기, 분기보고서, 보고서, 2017.03]
7                            [기타, 시장, 안내, 업종, 업종변경, 변경]
4        [단일, 판매, ㆍ, 공급, 계약, 체결, 자회사, 주요, 경영, 경영사항, 사항]
2                              [풍문, 보도, 해명, 미, 미확정, 확정]
6             [영업, 영업양수, 양수, 결정, 자회사, 주요, 경영, 경영

In [ ]:
spacing = Spacing()
df_a['공시보고서명'] = df_a['공시보고서명'].apply(lambda x : spacing(x))
df_a

## 완성코드

In [68]:
# 한글,영어만 남기기
def clean_text(docs):
    docs = re.sub('[^가-힣A-Za-z ]', '', docs)
    docs = re.sub('\s+', ' ', docs)
    docs = '' if docs== ' ' else docs
    return docs

In [70]:
# 완성코드
db_list = pd.read_csv('./data/개별기업명_511.csv',index_col=0)
path = './data/공시_다트크롤링/'
file_list = os.listdir(path)
spacing = Spacing()

for file in file_list[:]:
    corp_name = file[:-4]
    print(corp_name)
    if corp_name in db_list['회사이름'].values:
        file_path = os.path.join(path,file)
        df = pd.read_csv(file_path,index_col=0)
        df = df[['접수일자','공시대상회사','보고서명']]
        df.columns = ['날짜','공시회사','공시보고서명']
        df['날짜'] = pd.to_datetime(df['날짜'])
        df['날짜'] = df['날짜'].apply(lambda x : x.to_period(freq='D'))
        df.drop_duplicates(subset=['공시보고서명'], inplace=True)
        df = df.sort_values(by='날짜')
        
        df['공시보고서명'] = df['공시보고서명'].apply(clean_text)
        df['공시보고서명'] = df['공시보고서명'].apply(spacing)
        df['공시보고서명'] = df['공시보고서명'].apply(han.nouns)
        df.to_csv('./data/개별기업공시_전처리/{}.csv'.format(corp_name))


AJ네트웍스
AK홀딩스
BGF
BGF리테일
BNK금융지주
BYC
CJ CGV
CJ
CJ대한통운
CJ씨푸드
CJ제일제당
CS홀딩스
DB
DB금융투자
DB손해보험
DB하이텍
DGB금융지주
DL
DL건설
DL이앤씨
DRB동일
DSR
DSR제강
E1
ESR켄달스퀘어리츠
F&F 홀딩스
F&F
GKL
GS
GS건설
GS글로벌
GS리테일
HDC
HDC랩스
HDC현대EP
HDC현대산업개발
HD현대
HJ중공업
HLB글로벌
HMM
HSD엔진
JB금융지주
JW생명과학
JW중외제약
JW홀딩스
KB금융
KC그린홀딩스
KC코트렐
KEC
KG스틸
KG케미칼
KH 필룩스
KISCO홀딩스
KPX케미칼
KPX홀딩스
KR모터스
KSS해운
KTcs
KTis
LF
LG
LG디스플레이
LG생활건강
LG에너지솔루션
LG유플러스
LG이노텍
LG전자
LG헬로비전
LG화학
LIG넥스원
LS
LS네트웍스
LS전선아시아
LX인터내셔널
LX하우시스
LX홀딩스
MH에탄올
NAVER
NH올원리츠
NH투자증권
NH프라임리츠
NICE
NI스틸
OCI
PI첨단소재
POSCO홀딩스
S-Oil
SBS
SGC에너지
SG글로벌
SG세계물산
SHD
SH에너지화학
SIMPAC
SK
SKC
SK가스
SK네트웍스
SK디스커버리
SK디앤디
SK렌터카
SK리츠
SK바이오사이언스
SK스퀘어
SK아이이테크놀로지
SK이노베이션
SK증권
SK케미칼
SK텔레콤
SK하이닉스
SNT모티브
SNT에너지
SNT중공업
SNT홀딩스
SPC삼립
STX
STX엔진
STX중공업
SUN&L
TCC스틸
TYM
WISCOM
YG PLUS
가온전선
강남제비스코
강원랜드
갤럭시아에스엠
경농
경동나비엔
경동도시가스
경동인베스트
경방
경보제약
경인양행
경인전자
계룡건설산업
계양전기
고려산업
고려아연
고려제강
광동제약
광명전기
광전자
광주신세계
교보증권
교촌에프앤비
국도화학
국동
국보
국제약품
그린케미칼
극동유화
금강공업
금비
금양
금호건설
금호석유화학
금호에이치티
금호전기
금호타이어
기신정기
기아
기업은행
까뮤이앤씨
깨끗한나라
남광토건
남선알미늄
남성
남양유업
남해화학
넥센

## 누락된 40개 기업 포함 다시 크롤링

In [5]:
# 한글,영어만 남기기
def clean_text(docs):
    docs = re.sub('[^가-힣A-Za-z ]', '', docs)
    docs = re.sub('\s+', ' ', docs)
    docs = '' if docs== ' ' else docs
    return docs

In [6]:
from pykospacing import Spacing
from konlpy.tag import Hannanum,Kkma,Komoran
han = Hannanum()
spacing = Spacing()

In [11]:
db_list = pd.read_csv('./data/개별기업명_511.csv',index_col=0)
path = './data/공시_다트크롤링/'
file_list = os.listdir(path)
spacing = Spacing()

for file in file_list[:]:
    corp_name = file[:-4]
    if corp_name in db_list['회사이름'].values:
        print(corp_name)
        file_path = os.path.join(path,file)
        df = pd.read_csv(file_path,index_col=0)
        df = df[['접수일자','공시대상회사','보고서명']]
        df.columns = ['날짜','공시회사','공시보고서명']
        df['날짜'] = pd.to_datetime(df['날짜'])
        df['날짜'] = df['날짜'].apply(lambda x : x.to_period(freq='D'))
        df.drop_duplicates(inplace=True)
        df = df.sort_values(by='날짜')
        
        df['공시보고서명'] = df['공시보고서명'].apply(clean_text)
        df['공시보고서명'] = df['공시보고서명'].apply(spacing)
        df['공시보고서명'] = df['공시보고서명'].apply(han.nouns)
        df.to_csv('./data/개별기업공시_전처리/{}.csv'.format(corp_name))

In [8]:
import tensorflow as tf
tf.__version__

'2.6.4'